In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import json

import numpy as np
import pandas as pd
from urllib.request import urlopen

from tqdm import tqdm
import multiprocessing

#pd.set_option('display.max_colwidth', None)

# Obtención del header de las tesis

In [ ]:
intervals = list(range(0, 20347, 10))
print(intervals)

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470, 1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, 1580, 1590, 1600, 1610, 1620, 1630, 1640, 1650, 1660, 1670, 1680, 1690, 1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790, 1800, 1810, 1820, 1830, 1840, 

In [ ]:
data_rows = []

for i in tqdm(intervals):

    main_url = "https://renati.sunedu.gob.pe/handle/sunedu/191/simple-search?query=&filter_field_1=dateIssued&filter_type_1=equals&filter_value_1=%5B2000+TO+2024%5D&sort_by=score&order=desc&rpp=10&etal=0&start=" + str(i)
    html = urlopen(main_url)
    soup = BeautifulSoup(html, 'html.parser')

    tesis = soup.find_all("div", {"class":"row item-list-item"})

    for t in tesis:
        try:
            thesis_type = t.find("div", {"field-type"}).text
            title = t.find("div", {"field-title"}).find("a").text
            url = t.find("div", {"field-title"}).find("a").get("href")
            url = "https://renati.sunedu.gob.pe" + url
            author = t.find_all("div", {"field-contributor"})[0].find("a").text
            advisor = t.find_all("div", {"field-contributor"})[1].find("em").text
            year = t.find("div", {"field-date"}).text
            university = t.find("div", {"field-publisher"}).text
            right = t.find("div", {"field-rights"}).find("img").get("title")
            keyword = t.find("div", {"field-subject"}).text

        except:
            thesis_type = None
            title = None
            url = None
            url = None
            author = None
            advisor = None
            year = None
            university = None
            right = None
            keyword = None

        data = {"thesis_type": thesis_type,
                "title": title,
                "url": url,
                "author": author,
                "advisor": advisor,
                "year": year,
                "university": university,
                "right": right,
                "keyword": keyword}

        data_rows.append(data)

    break

len(data_rows)

  0%|          | 0/2035 [00:01<?, ?it/s]


10

In [ ]:
len(data_rows)

19140

## Almacenamiento de los datos

In [ ]:
# with open("thesis_unmsm.json", "w") as f:
#     json.dump(data_rows, f, indent=4)

In [ ]:
f = open("thesis_unmsm.json", "r")
thesis_json = json.load(f)
f.close()

len(thesis_json)

19140

## Procesamiento de los datos

In [ ]:
## conversión de JSON a PANDAS
df_thesis = pd.DataFrame(thesis_json)
df_thesis.head(1)

,thesis_type,title,url,author,advisor,year,university,right,keyword
0,info:eu-repo/semantics/bachelorThesis,Análisis hidrogeológico conceptual y propuesta...,https://renati.sunedu.gob.pe/handle/sunedu/316...,"Contreras Gutiérrez, Efrain Jhiaromir","Mogrovejo Gutiérrez, Rubén Esaú",2022,Universidad Nacional Mayor de San Marcos,info:eu-repo/semantics/openAccess,Mecánica de suelos; Aguas subterráneas


In [ ]:
df_thesis.isna().sum()

thesis_type    3748
title          3748
url            3748
author         3748
advisor        3748
year           3748
university     3748
right          3748
keyword        3748
dtype: int64

In [ ]:
## elimación de valores nulos
df_thesis_pre = df_thesis.dropna().copy()

## preprocesamiento de columnas
df_thesis_pre["thesis_type"] = df_thesis_pre["thesis_type"].apply(lambda x: x.split("eu-repo/semantics/")[1])
df_thesis_pre["title"] = df_thesis_pre["title"].apply(lambda x: x.replace("\r\n", " "))
df_thesis_pre["id"] = df_thesis_pre["url"].apply(lambda x: x.split("handle/sunedu/")[1])
df_thesis_pre["right"] = df_thesis_pre["right"].apply(lambda x: x.split("eu-repo/semantics/")[1])
df_thesis_pre["keyword"] = df_thesis_pre["keyword"].apply(lambda x: x.replace(";", ","))

## ordenamiento de las colunmnas
df_thesis_pre = df_thesis_pre[['id', 'thesis_type', 'title', 'url', 'author', 'advisor',
                               'year', 'university', 'right', 'keyword']].copy()

print(df_thesis_pre.shape)
df_thesis_pre.sample(3)

(15392, 10)


,id,thesis_type,title,url,author,advisor,year,university,right,keyword
13890,2875574,bachelorThesis,Anglicismos difundidos por la prensa y la publ...,https://renati.sunedu.gob.pe/handle/sunedu/287...,"Reyes Padilla, Víctor Antolín","Chávez Reyes, Amancio",2009,Universidad Nacional Mayor de San Marcos,openAccess,Español - Términos y frases de origen extranje...
11610,2884055,bachelorThesis,Perfil sociocultural y características del uso...,https://renati.sunedu.gob.pe/handle/sunedu/288...,"Almerco Rojas, Delsi Nataly","Pante Salas, Giovanna Glagys",2018,Universidad Nacional Mayor de San Marcos,closedAccess,Adolescentes - Conducta sexual - Perú - Lima (...
17157,3447674,bachelorThesis,Actitud frente a la eutanasia y sus factores a...,https://renati.sunedu.gob.pe/handle/sunedu/344...,"Tena Valladolid, Milagros Valeria","Ocampo Zegarra, Juan Carlos",2023,Universidad Nacional Mayor de San Marcos,openAccess,Universidad Nacional Mayor de San Marcos (Lima...


In [ ]:
print(df_thesis_pre.shape)
print(df_thesis_pre["id"].nunique())
print(df_thesis_pre["title"].nunique())

(15392, 10)
15392
15320


In [ ]:
df_thesis_pre[df_thesis_pre["id"]=="2863779"] ## 61 datos con ;

,id,thesis_type,title,url,author,advisor,year,university,right,keyword
212,2863779,masterThesis,Modelo de estimación de riesgos ergonómicos en...,https://renati.sunedu.gob.pe/handle/sunedu/286...,"Aguilar Lugo, Amalia","Chávez Machado, Jesús Ángel Alejandro",2013,Universidad Nacional Mayor de San Marcos,closedAccess,"Salud ocupacional, Mineros - Enfermedades - Pe..."


In [ ]:
df_thesis_pre[df_thesis_pre["id"]=="2866421"] ## advisor separado por ;

,id,thesis_type,title,url,author,advisor,year,university,right,keyword
48,2866421,bachelorThesis,Percepción del usuario sobre la calidad de ate...,https://renati.sunedu.gob.pe/handle/sunedu/286...,"Lupaca Layme, Pedro Sebastian","Mendoza Solis, Olga Luz; Durand Barreto, Juana...",2014,Universidad Nacional Mayor de San Marcos,openAccess,"Percepción, Atención hospitalaria-Evaluación, ..."


In [ ]:
df_thesis_pre.to_csv("thesis_unmsm.csv", index=False, sep="|")

# Obtención del detalle de las tesis

In [ ]:
df_thesis_pre = pd.read_csv("thesis_unmsm.csv", sep="|")
print(df_thesis_pre.shape)
df_thesis_pre.head(1)

(15392, 10)


,id,thesis_type,title,url,author,advisor,year,university,right,keyword
0,3161802,bachelorThesis,Análisis hidrogeológico conceptual y propuesta...,https://renati.sunedu.gob.pe/handle/sunedu/316...,"Contreras Gutiérrez, Efrain Jhiaromir","Mogrovejo Gutiérrez, Rubén Esaú",2022,Universidad Nacional Mayor de San Marcos,openAccess,"Mecánica de suelos, Aguas subterráneas"


In [ ]:
ids = df_thesis_pre["id"].unique()
len(ids)

15392

In [ ]:
ids[7999:8010]

array([2865861, 2865775, 2865933, 2865762, 2866355, 2866436, 2866398,
       2866407, 2866908, 2866951, 2866941])

In [ ]:
def get_thesis_detail(id):
    url = "https://renati.sunedu.gob.pe/handle/sunedu/" + str(id)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    data = {}
    data["id"] = str(id)

    ## obtención de la metadata
    metadata = soup.find_all("div", {"class":"metadataField"})
    for m in metadata:
        try:
            key = m.find("span", {"class":"metadataFieldLabel"}).text
            value = m.find("span", {"class":"metadataFieldValue"}).text

        except:
            key = "None"
            value = "None"

        data[key] = value

    return data

## creación del grupo de procesos
pool = multiprocessing.Pool()

## obtención de la metada en paralelo
# all_details = pool.map(get_thesis_detail, ids[0:10])
ids = ids[10000:]
all_details = []
v = 10001
for data in tqdm(pool.imap(get_thesis_detail, ids), total=len(ids)):
    all_details.append(data)

    ## almacenamiento de los resultados cada 2000 iteraciones
    if v % 2000 == 0:
        with open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_unmsm_{}.json".format(v), "w") as f:
            json.dump(all_details, f, indent=4)

    v += 1

pool.close()
pool.join()

len(all_details)

100%|██████████| 5392/5392 [1:29:21<00:00,  1.01it/s]


5392

In [ ]:
with open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_unmsm_16000.json", "w") as f:
            json.dump(all_details, f, indent=4)

## Unión de todas las partes de la metadata extraída

In [ ]:
f = open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_unmsm_2000.json", "r")
json_2000 = json.load(f)
print(len(json_2000))
f.close()

f = open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_unmsm_4000.json", "r")
json_4000 = json.load(f)
print(len(json_4000))
f.close()

f = open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_unmsm_6000.json", "r")
json_6000 = json.load(f)
print(len(json_6000))
f.close()

f = open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_unmsm_8000.json", "r")
json_8000 = json.load(f)
print(len(json_8000))
f.close()

f = open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_unmsm_10000.json", "r")
json_10000 = json.load(f)
print(len(json_10000))
f.close()

f = open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_unmsm_16000.json", "r")
json_16000 = json.load(f)
print(len(json_16000))
f.close()

2000
4000
2000
4000
2000
5392


In [ ]:
all_json = json_4000 + json_8000 + json_10000 + json_16000
print(len(all_json))

ids_val = []
for i in all_json:
    ids_val.append(i["id"])

print(len(ids_val))
print(len(set(ids_val)))

15392
15392
15392


In [ ]:
thesis_details_json = json_4000 + json_8000 + json_10000 + json_16000
len(thesis_details_json)

15392

In [ ]:
with open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_json.json", "w") as f:
    json.dump(thesis_details_json, f, indent=4)

In [ ]:
f = open("/content/drive/MyDrive/UPC/MAESTRIA_DS/8. Redes Complejas/Data Scraping/thesis_details_json.json", "r")
thesis_details_json = json.load(f)
f.close()

len(thesis_details_json)

15392

## Conversión a Pandas Dataframe

In [ ]:
thesis_details_json[0]

{'id': '3161802',
 'Title:\xa0': 'Análisis hidrogeológico conceptual y propuesta de escenarios numéricos del interflujo para la presa de relaves ubicada en la ciudad de Castilla Arequipa-2022',
 'Authors(s):\xa0': 'Contreras Gutiérrez, Efrain Jhiaromir',
 'Advisor(s):\xa0': 'Mogrovejo Gutiérrez, Rubén Esaú',
 'Keywords:\xa0': 'Mecánica de suelos;\xa0Aguas subterráneas',
 'OCDE field:\xa0': 'https://purl.org/pe-repo/ocde/ford#2.03.01',
 'Issue Date:\xa0': '2022',
 'Institution:\xa0': 'Universidad Nacional Mayor de San Marcos',
 'Abstract:\xa0': 'Se realizó el estudio de mecánica de suelos y mecánica de rocas, mediante el estudio geo mecánico, acudiendo a ensayos de laboratorio, en la presa de relaves ubicada en el departamento de Arequipa, provincia de Castilla, distrito de Orcopampa. Se determinó el tipo de suelo y el tipo de roca (Dacita), realizando el perfil estratigráfico, para determinar; las filtraciones en la zona de las recargas y en la presa de relaves; asimismo, se realizó el

In [ ]:
data_rows = []
for t in thesis_details_json:
    try:
        id = t["id"]
    except:
        id = None

    try:
        title = t["Title:\xa0"]
    except:
        title = None

    try:
        author = t["Authors(s):\xa0"]
    except:
        author = None

    try:
        advisor = t["Advisor(s):\xa0"]
    except:
        advisor = None

    try:
        keywords =  t["Keywords:\xa0"]
    except:
        keywords = None

    try:
        ocde = t["OCDE field:\xa0"],
    except:
        ocde = None

    try:
        date = t["Issue Date:\xa0"]
    except:
        date = None

    try:
        university = t["Institution:\xa0"]
    except:
        university = None

    try:
        abstract = t["Abstract:\xa0"]
    except:
        abstract = None

    try:
        discipline = t["Discipline:\xa0"]
    except:
        discipline = None

    try:
        grade = t["Grade or title grantor:\xa0"]
    except:
        grade = None

    try:
        juror = t["Juror:\xa0"]
    except:
        juror = None

    try:
        register_date = t["Register date:\xa0"]
    except:
        register_date = None

    data = {"id": id,
            "title": title,
            "author": author,
            "advisor": advisor,
            "keywords": keywords,
            "ocde": ocde,
            "date": date,
            "university": university,
            "abstract": abstract,
            "discipline": discipline,
            "grade": grade,
            "juror": juror,
            "register_date": register_date}

    data_rows.append(data)

print(len(data_rows))
data_rows[0]

15392


{'id': '3161802',
 'title': 'Análisis hidrogeológico conceptual y propuesta de escenarios numéricos del interflujo para la presa de relaves ubicada en la ciudad de Castilla Arequipa-2022',
 'author': 'Contreras Gutiérrez, Efrain Jhiaromir',
 'advisor': 'Mogrovejo Gutiérrez, Rubén Esaú',
 'keywords': 'Mecánica de suelos;\xa0Aguas subterráneas',
 'ocde': ('https://purl.org/pe-repo/ocde/ford#2.03.01',),
 'date': '2022',
 'university': 'Universidad Nacional Mayor de San Marcos',
 'abstract': 'Se realizó el estudio de mecánica de suelos y mecánica de rocas, mediante el estudio geo mecánico, acudiendo a ensayos de laboratorio, en la presa de relaves ubicada en el departamento de Arequipa, provincia de Castilla, distrito de Orcopampa. Se determinó el tipo de suelo y el tipo de roca (Dacita), realizando el perfil estratigráfico, para determinar; las filtraciones en la zona de las recargas y en la presa de relaves; asimismo, se realizó el análisis de estabilidad de la presa de relaves, en la se

In [ ]:
df_thesis_details = pd.DataFrame(data_rows)
print(df_thesis_details.shape)
df_thesis_details.head(1)

(15392, 13)


,id,title,author,advisor,keywords,ocde,date,university,abstract,discipline,grade,juror,register_date
0,3161802,Análisis hidrogeológico conceptual y propuesta...,"Contreras Gutiérrez, Efrain Jhiaromir","Mogrovejo Gutiérrez, Rubén Esaú",Mecánica de suelos; Aguas subterráneas,"(https://purl.org/pe-repo/ocde/ford#2.03.01,)",2022,Universidad Nacional Mayor de San Marcos,Se realizó el estudio de mecánica de suelos y ...,Ingeniería Mecánica de Fluidos,Universidad Nacional Mayor de San Marcos. Facu...,"Serruto Colque, Arístides Ramón; Sarango Julca...",14-Jun-2022; 14-Jun-2022


In [ ]:
df_thesis_details.isna().sum()

id                  0
title               0
author              0
advisor             0
keywords            0
ocde                0
date                0
university          0
abstract            0
discipline          0
grade               0
juror            6570
register_date       0
dtype: int64

In [ ]:
print(df_thesis_details["id"].nunique())
print(df_thesis_details["title"].nunique())

15392
15320


## Merge datasets

In [ ]:
print(df_thesis_pre.shape)
df_thesis_pre["id"] = df_thesis_pre["id"].astype(str)

df_thesis_all = pd.merge(df_thesis_pre[["id", "thesis_type", "right", "url"]], df_thesis_details,
                         how="left",
                         on="id")

print(df_thesis_all.shape)

(15392, 10)
(15392, 16)


In [ ]:
df_thesis_all.head()

,id,thesis_type,right,url,title,author,advisor,keywords,ocde,date,university,abstract,discipline,grade,juror,register_date
0,3161802,bachelorThesis,openAccess,https://renati.sunedu.gob.pe/handle/sunedu/316...,Análisis hidrogeológico conceptual y propuesta...,"Contreras Gutiérrez, Efrain Jhiaromir","Mogrovejo Gutiérrez, Rubén Esaú",Mecánica de suelos; Aguas subterráneas,"(https://purl.org/pe-repo/ocde/ford#2.03.01,)",2022,Universidad Nacional Mayor de San Marcos,Se realizó el estudio de mecánica de suelos y ...,Ingeniería Mecánica de Fluidos,Universidad Nacional Mayor de San Marcos. Facu...,"Serruto Colque, Arístides Ramón; Sarango Julca...",14-Jun-2022; 14-Jun-2022
1,3161807,doctoralThesis,openAccess,https://renati.sunedu.gob.pe/handle/sunedu/316...,La regulación democrática del financiamiento p...,"Manrique Prialé, Roxana Marlene","Chanamé Orbe, Raúl Roosevelt",Jurado Nacional de Elecciones; Financiamiento;...,(https://purl.org/pe-repo/ocde/ford#5.06.01; h...,2022,Universidad Nacional Mayor de San Marcos,Define la relación que existe entre el financi...,Derecho y Ciencia Política,Universidad Nacional Mayor de San Marcos. Facu...,"Paiva Goyburu, Dante Martín; Paucar Chappa, Ma...",15-Jun-2022; 15-Jun-2022
2,3161831,masterThesis,openAccess,https://renati.sunedu.gob.pe/handle/sunedu/316...,Sistema de gestión de prevención de riesgos la...,"Godoy Martínez, Manuel Rómulo","Tinoco Gómez, Oscar Rafael",Sistemas de gestión; Seguridad y salud en el t...,"(https://purl.org/pe-repo/ocde/ford#2.11.04,)",2022,Universidad Nacional Mayor de San Marcos,Demostrar que implementar un plan para el sist...,Dirección de Empresas Industriales y de Servicios,Universidad Nacional Mayor de San Marcos. Facu...,"Rivera Poma, Juan Manuel; Inche Mitma, Jorge L...",17-Jun-2022; 17-Jun-2022
3,3161838,bachelorThesis,openAccess,https://renati.sunedu.gob.pe/handle/sunedu/316...,Análisis del uso de tecnologías de información...,"Briones Heras, Mauricio","Pantoja Collantes, Jorge Santiago",Elecciones - Innovaciones tecnológicas - Perú;...,"(https://purl.org/pe-repo/ocde/ford#2.02.04,)",2018,Universidad Nacional Mayor de San Marcos,Describe el uso de las TICs para abordar el pr...,Ingeniería de Sistemas,Universidad Nacional Mayor de San Marcos. Facu...,"Pró Concepción, Luzmila Elisa; Román Concha, N...",16-Jun-2022; 16-Jun-2022
4,3161822,bachelorThesis,openAccess,https://renati.sunedu.gob.pe/handle/sunedu/316...,Eficacia de los programas de rehabilitación en...,"Paucar Arellano, Catherine Isabel","Mendoza Correa, María Isabel del Milagro",Actividad motora; Pérdida auditiva; Rehabilita...,(https://purl.org/pe-repo/ocde/ford#3.05.03; h...,2022,Universidad Nacional Mayor de San Marcos,Sistematiza los artículos sobre la eficacia de...,Tecnología Médica en el área de Terapia Física...,Universidad Nacional Mayor de San Marcos. Facu...,"Garay Ugaz, Eduardo Martin; Gómez Gonzales, Wa...",3-Jun-2022; 3-Jun-2022


## Procesamiento de los datos

### Remoción de duplicados

In [ ]:
print(df_thesis_all["title"].count())
print(df_thesis_all["title"].nunique())

15392
15320


In [ ]:
ids_duplicated = df_thesis_all[df_thesis_all["title"].duplicated()]["id"].unique()
len(ids_duplicated)

72

In [ ]:
df_thesis_all_pre = df_thesis_all[~df_thesis_all["id"].isin(ids_duplicated)].copy()
print(df_thesis_all_pre.shape)

(15320, 16)


## Almacenamiento de los datos procesados

In [ ]:
df_thesis_all_pre.to_csv("df_thesis_all.csv", index=False, sep="|")